In [ ]:
from spf.dataset.spf_dataset import v5spfdataset

ds = v5spfdataset(
    "/mnt/md0/spf/2d_wallarray_v2_data/june/wallarrayv3_2024_06_03_01_17_43_nRX2_rx_circle.zarr",
    nthetas=65,
    ignore_qc=True,
    precompute_cache="/home/mouse9911/precompute_cache_chunk16_fresh/",
    gpu=True,
    snapshots_per_session=1,
)

In [ ]:
ds[0][0]["signal_matrix"][0].numpy().shape

In [ ]:
from spf.rf import beamformer_given_steering_nomean, get_phase_diff, simple_segment

default_segment_args = {
    "window_size": 2048,
    "stride": 2048,
    "trim": 20.0,
    "mean_diff_threshold": 0.2,
    "max_stddev_threshold": 0.5,
    "drop_less_than_size": 3000,
    "min_abs_signal": 40,
}
simple_segment(ds[0][0]["signal_matrix"][0].numpy(), **default_segment_args)

In [ ]:
from spf.rf import beamformer_given_steering_nomean, get_phase_diff, simple_segment
import matplotlib.pyplot as plt
import numpy as np

session_idx = 0
data = ds[session_idx][0]
# signal_matrix = load_zarr_to_numpy(z.receivers["r0"].signal_matrix[session_idx])
n = 2 * 4 * 50000
offset = 50000
v = data["signal_matrix"][0, :, offset : offset + n].numpy()
pd = get_phase_diff(v)

fig, axs = plt.subplots(2, 1, figsize=(12, 6))

axs[0].scatter(np.arange(n), np.abs(v[0]), alpha=0.1, s=1, label="ant0")
axs[0].scatter(np.arange(n), np.abs(v[1]), alpha=0.1, s=1, label="ant1")
axs[0].set_title("Raw signal")
axs[0].legend()
axs[0].set_xlabel("Sample# (time)")
axs[1].set_xlabel("Sample# (time)")
axs[1].set_title("Phase estimates")
axs[1].scatter(np.arange(n), pd, s=1, alpha=0.1)

fig.set_tight_layout(True)

In [ ]:
beam_sds = [
    beamformer_given_steering_nomean(
        steering_vectors=ds.steering_vectors[receiver_idx],
        signal_matrix=v,
    )
    for receiver_idx in range(2)
]

window_sds = []
for window in simple_segment(
    v,
    window_size=2500,
    stride=2500,
    trim=20,
    mean_diff_threshold=0.2,  #
    max_stddev_threshold=0.5,  # just eyeballed this
    drop_less_than_size=3000,
    min_abs_signal=40,
)["simple_segmentation"]:
    if window["type"] == "signal":
        axs[1].plot(
            [window["start_idx"], window["end_idx"]],
            [window["mean"], window["mean"]],
            color="red",
        )
    else:
        axs[1].plot(
            [window["start_idx"], window["end_idx"]],
            [window["mean"], window["mean"]],
            color="orange",
        )
    # print(window["start_idx"], window["end_idx"])
    _beam_sds = beam_sds[0][:, window["start_idx"] : window["end_idx"]].mean(axis=1)
    # _beam_sds = _beam_sds.mean(axis=1)
    # _beam_sds -= _beam_sds.min()
    window_sds.append(_beam_sds)
window_sds = np.array(window_sds)

In [ ]:
v

In [ ]:
beamformer_given_steering_nomean(
    steering_vectors=ds.steering_vectors[0],
    signal_matrix=v,
)